In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

c:\Users\Aditi Mathur\Documents\AI Workspace\PyJHora\src


In [ ]:
from jhora import const, utils
from jhora.panchanga import drik
from jhora.horoscope.chart import  charts
from jhora.horoscope.chart import house


c:\Users\Aditi Mathur\Documents\AI Workspace\PyJHora added to system path ['C:\\Users\\Aditi Mathur\\.conda\\envs\\py310\\python310.zip', 'C:\\Users\\Aditi Mathur\\.conda\\envs\\py310\\DLLs', 'C:\\Users\\Aditi Mathur\\.conda\\envs\\py310\\lib', 'C:\\Users\\Aditi Mathur\\.conda\\envs\\py310', 'c:\\Users\\Aditi Mathur\\Documents\\AI Workspace\\PyJHora\\venv', '', 'c:\\Users\\Aditi Mathur\\Documents\\AI Workspace\\PyJHora\\venv\\lib\\site-packages', 'c:\\Users\\Aditi Mathur\\Documents\\AI Workspace\\PyJHora\\venv\\lib\\site-packages\\win32', 'c:\\Users\\Aditi Mathur\\Documents\\AI Workspace\\PyJHora\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Aditi Mathur\\Documents\\AI Workspace\\PyJHora\\venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\Aditi Mathur\\Documents\\AI Workspace\\PyJHora\\src', 'c:\\Users\\Aditi Mathur\\Documents\\AI Workspace\\PyJHora']


In [3]:
import json

In [4]:
_lang_path = const._LANGUAGE_PATH

In [5]:
def get_prediction_resources(language='en'):
    """
        get resources from prediction_msgs_<lang>.txt
        @param language: Two letter language code. en, hi, ka, ta, te
        @return json strings from the resource file as dictionary 
    """
    json_file = _lang_path + const._DEFAULT_PREDICTION_JSON_FILE_PREFIX+language+'.json'
    print('JSON_FILE_PATH', json_file)
    f = open(json_file,"r",encoding="utf-8")
    msgs = json.load(f)
    return msgs

In [6]:
def _get_general_lagna_rasi_prediction(jd,place,prediction_msgs,language=const._DEFAULT_LANGUAGE):
    janma_rasi = drik.raasi(jd, place)[0]-1
    results = {}
    source_count = 2
    for s in range(source_count):
        ks = utils.resource_strings['janma_rasi_str']+'_'+str(s+1)
        results[ks] = "<html><b>"+utils.resource_strings['general_prediction_str']+"</b><br>"
        #results[ks] += "<b>"+prediction_msgs['general_prediction_caution']+"</b><br>"
        results[ks] += "<b>"+prediction_msgs['janma_raasi_'+str(s+1)]['source']+"</b><br>"
        pdict = prediction_msgs['janma_raasi_'+str(s+1)][str(janma_rasi+1)]
        for k,v in pdict.items():
            results[ks] += "<b>"+k+"</b><br>"+v+"<br>"
    return results

In [7]:
PLANET_NAMES = """Sun☉,Moon☾,Mars♂,Mercury☿,Jupiter♃,Venus♀,Saturn♄,Raagu☊,Kethu☋,Uranus⛢,Neptune♆,Pluto♇""".split(',')

In [8]:
def _get_planets_in_houses_prediction(planet_positions,prediction_msgs):
    p_to_h = utils.get_planet_house_dictionary_from_planet_positions(planet_positions)
    lagna_house = p_to_h['L']
    ks = utils.resource_strings['planets_str']
    results = {}
    results[ks] = "<html>"#<b>"+ks+"</b><br>"
    #results[ks] += "<b>"+utils.resource_strings['general_prediction_caution']+"</b><br>"
    planet_msgs = prediction_msgs['planets_in_houses']
    #print('planet msgs',planet_msgs)
    for planet in [*range(9)]:
        planet_house = house.get_relative_house_of_planet(lagna_house,p_to_h[planet])
        pl_msg = planet_msgs[str(planet_house)][planet]
        #print(planet,planet_house,pl_msg)
        key = PLANET_NAMES[planet]+'-'+utils.resource_strings['house_str']+'#'+str(planet_house)+":"
        results[ks] += "<b>"+key+"</b><br>"+pl_msg+"<br>"
    return results
def _get_lords_in_houses_prediction(planet_positions,prediction_msgs):
    p_to_h = utils.get_planet_house_dictionary_from_planet_positions(planet_positions)
    lagna_house = p_to_h['L']
    ks = utils.resource_strings['houses_str']
    results = {}
    results[ks] = "<html>"#<b>"+ks+"</b><br>"
    #results[ks] += "<b>"+utils.resource_strings['general_prediction_caution']+"</b><br>"
    planet_msgs = prediction_msgs['lord_of_a_house_joining_lord_of_another_house']
    #print('planet msgs',planet_msgs)
    for h in [*range(12)]:
        lord = const._house_owners_list[(h+lagna_house)%12]
        house_of_lord = house.get_relative_house_of_planet(lagna_house,p_to_h[lord])
        key = "Lord of House#"+str(h+1)+" in house#"+str(house_of_lord)
        #print('key',key)
        pl_msg = planet_msgs[str(h+1)][house_of_lord-1]
        results[ks] += "<b>"+key+"</b><br>"+pl_msg+"<br>"
    return results

In [9]:
@staticmethod
def _get_location(place_name):
    result = utils.get_location(place_name)
    print('RESULT',result)
    if result:
        _place_name,_latitude,_longitude,_time_zone = result
        # _place_text.setText(_place_name)
        # _lat_text.setText(str(_latitude))
        # _long_text.setText(str(_longitude))
        # _tz_text.setText(str(_time_zone))
        print(_place_name,_latitude,_longitude,_time_zone)
        return _place_name,_latitude,_longitude,_time_zone
    else:
        msg = place_name+" could not be found in OpenStreetMap.\nTry entering latitude and longitude manually.\nOr try entering nearest big city"
        print(msg)
        return None
        # QMessageBox.about(self,"City not found",msg)
    #     _lat_text.setText('')
    #     _long_text.setText('')
    # _reset_place_text_size()

In [10]:
def get_prediction_details(jd_at_dob,place,language=const._DEFAULT_LANGUAGE):
    prediction_msgs = get_prediction_resources(language=language)
    print('prediction keys',prediction_msgs.keys())
    results = {}
    planet_positions = charts.rasi_chart(jd_at_dob, place)
    results1 = _get_general_lagna_rasi_prediction(jd_at_dob,place,prediction_msgs,language=language)
    print('results1',results1)
    results.update(results1)
    results2 = _get_planets_in_houses_prediction(planet_positions,prediction_msgs)
    results.update(results2)
    results3 = _get_lords_in_houses_prediction(planet_positions,prediction_msgs)
    results.update(results3)
    return results

In [11]:
def get_prediction_resources(language='en'):
    """
        get resources from prediction_msgs_<lang>.txt
        @param language: Two letter language code. en, hi, ka, ta, te
        @return json strings from the resource file as dictionary 
    """
    json_file = _lang_path + const._DEFAULT_PREDICTION_JSON_FILE_PREFIX+language+'.json'
    print('JSON_FILE_PATH', json_file)
    f = open(json_file,"r",encoding="utf-8")
    msgs = json.load(f)
    return msgs

In [12]:
year=1992
month=8
day=1
_tob_text="3:10:00"

In [13]:
birth_date = drik.Date(int(year),int(month),int(day))
# user_age = min(datetime.now().year - birth_date.year, const.annual_maximum_age) + 1
dob = (int(year),int(month),int(day))
tob = tuple([int(x) for x in _tob_text.split(':')])
print('*DOB', dob)
print('*TOB', tob)


*DOB (1992, 8, 1)
*TOB (3, 10, 0)


In [14]:
_birth_julian_day = utils.julian_day_number(dob, tob)
print('*Julian Day', _birth_julian_day)


*Julian Day 2448835.6319444445


In [15]:
_place_name,_latitude,_longitude,_time_zone= _get_location('Jaipur, India')

Jaipur, India in the database
RESULT ['Jaipur, India', 26.9196, 75.7878, 5.5]
Jaipur, India 26.9196 75.7878 5.5


In [16]:
place = drik.Place(_place_name,_latitude,_longitude,_time_zone)
print('*Place', place)
        

*Place Place(Place='Jaipur, India', latitude=26.9196, longitude=75.7878, timezone=5.5)


In [17]:
results = get_prediction_details(_birth_julian_day,place,language='en')

JSON_FILE_PATH c:\Users\Aditi Mathur\Documents\AI Workspace\PyJHora\src\jhora/lang/prediction_msgs_en.json
prediction keys dict_keys(['general_prediction_caution', 'lord_of_a_house_joining_lord_of_another_house', 'janma_raasi_1', 'janma_raasi_2', 'planets_in_houses'])
results1 {'Janma rasi_1': '<html><b>General Predictions</b><br><b>Source: How to judge a horoscope - BV. Raman</b><br><b>Mental Tendencies</b><br>People born in Leo are ambitious as well as avaricious, warm-hearted, and have a liking for art, literature and music. They are cheerful and unimpulsive.<br><b>Physical Tendencies</b><br>Persons born in this sign will be magnetic in appearance with broad shoulders, bilious constitution, of average height, oval faced, thoughtful countenance and the upper part of the body is generally better formed.<br><b>General Tendencies</b><br>They can adapt themselves to any condition in life. They have faith. In affection they are sincere. They stick to orthodox principles in religion but ar

In [18]:
results.keys()

dict_keys(['Janma rasi_1', 'Janma rasi_2', 'Planets', 'Houses'])

In [19]:
from IPython.display import display, HTML

In [20]:
HTML(results['Janma rasi_1'])

In [21]:
HTML(results['Janma rasi_2'])

In [22]:
HTML(results['Planets'])

In [23]:
HTML(results['Planets'])